## Test read in kraken2 output (without --use-mpa-style)
### From the results folder from nf-core https://nf-co.re/mag/results#mag/results-3b4dd3469725654d67e06b3853ba460d64c80788/


In [764]:
import pandas as pd
import altair as alt
import numpy as np
import pyfastx
alt.data_transformers.disable_max_rows()




DataTransformerRegistry.enable('default')

In [73]:
kraken_raw= pd.read_csv('/Users/williamrosenbaum/Downloads/kraken2_report.txt', 
                     sep='\t',
                     header=None,
                     names=['pct', 'reads_clade', 'reads_taxon', 'rank', 'ID', 'name'])

In [149]:
kraken = (kraken_raw
  # Find where the viruses starts in the dataframe 
 .loc[kraken_raw[lambda x: x['name'].str.contains('Viruses')].index[0]:, :]
 .loc[lambda x: x['reads_taxon'] > 10]
 .loc[lambda x: x['rank'].str.contains('S')]
 .assign(pct_reads=lambda x: x['reads_clade'] / x['reads_clade'].sum())
)

kraken


,pct,reads_clade,reads_taxon,rank,ID,name,pct_reads
13879,1.52,214053,214053,S,1211417,uncultured crAssphage,0.994393
14472,0.01,1117,1117,S1,1090134,Salmonella phage SPN3US,0.005189
14522,0.00,16,16,S,1273755,Halovirus HRTV-8,0.000074
15731,0.00,14,14,S,1051631,Streptococcus phage YMC-2011,0.000065
17776,0.00,17,17,S,50294,Psittacid alphaherpesvirus 1,0.000079
17779,0.00,14,14,S,10317,Cercopithecine alphaherpesvirus 2,0.000065
17936,0.00,29,29,S,2107708,Pandoravirus neocaledonia,0.000135


In [170]:
bar = alt.Chart(kraken).mark_bar().encode(
 alt.Y('name:N', sort='-x'),
 alt.X('reads_taxon:Q'),
 tooltip=[alt.Tooltip('name:N'),
          alt.Tooltip('reads_taxon:Q', title='Number of reads', format=',')]
)

text = bar.mark_text(
    align='left',
    dx=3
).encode(
    alt.Text('pct_reads:Q', format='.1%')
)

(bar + text)

alt.LayerChart(...)

## kraken (mpa-format from my pipeline)

In [284]:
kraken_megahit_raw = pd.read_csv('../results/kraken/megahit/APX.kraken2_megahit.report.tsv', 
                             sep='\t', 
                             header=None,
                             names=['taxonomy', 'count'])



def clean_kraken(df):
    return (df
        .assign(domain=lambda x: np.select([x['taxonomy'].str.contains('d__Viruses'),
                                            x['taxonomy'].str.contains('d__Bacteria')], 
                                                ['Virus', 
                                                'Bacteria'], 
                                            default='Other'))
        .assign(taxonomy=lambda x: x['taxonomy'].str.split('|'))
        .explode('taxonomy')
        .drop_duplicates(subset='taxonomy')
        .assign(kingdom=lambda x: x['taxonomy'].str.extract(r'k__(.*)'),
                phylum=lambda x: x['taxonomy'].str.extract(r'p__(.*)'),
                klass=lambda x: x['taxonomy'].str.extract(r'c__(.*)'),
                order=lambda x: x['taxonomy'].str.extract(r'o__(.*)'),
                family=lambda x: x['taxonomy'].str.extract(r'f__(.*)'),
                genus=lambda x: x['taxonomy'].str.extract(r'g__(.*)'),
                species=lambda x: x['taxonomy'].str.extract(r's__(.*)'))
        )



kraken_megahit = (clean_kraken(kraken_megahit_raw)
 .loc[lambda x: ~x['species'].isna()]
 .loc[lambda x: x['count'] > 100]
 .sort_values('count', ascending=False)
 .assign(percent=lambda x: x['count'] / x['count'].sum())
 .head(10)
)


kraken_metaspades = (clean_kraken(kraken_metaspades_raw)
 .loc[lambda x: ~x['species'].isna()]
 .loc[lambda x: x['count'] > 100]
 .sort_values('count', ascending=False)
 .assign(percent=lambda x: x['count'] / x['count'].sum())
 .head(10)
)



In [285]:
alt.Chart(kraken_megahit, title='Kraken megahit').mark_bar().encode(
 alt.X('percent:Q', axis=alt.Axis(format='.1%')),
 alt.Y('species:N', sort='-x'),
 alt.Color('species', title='Species')
).properties(width=500, height=500)

alt.Chart(...)

In [244]:
alt.Chart(kraken_metaspades, title='Kraken metaspades').mark_bar().encode(
 alt.X('percent:Q', axis=alt.Axis(format='.1%')),
 alt.Y('species:N', sort='-x'),
 alt.Color('species', title='Species')
).properties(width=500, height=500)

alt.Chart(...)

## KAIJU

In [286]:
kaiju_megahit_raw = pd.read_csv('../results/kaiju/megahit/APX_table_megahit.tsv', 
                    sep='\t')[['taxon_name', 'percent']]

kaiju_megahit = (kaiju_megahit_raw
 .assign(percent=lambda x: x['percent'] / 100)
 .sort_values('percent', ascending=False)
 .loc[lambda x: x['taxon_name'] != 'unclassified']
 .head(10)
)



In [287]:
alt.Chart(kaiju_megahit, title='Kaiju megahit').mark_bar().encode(
 alt.X('percent:Q', axis=alt.Axis(format='.1%')),
 alt.Y('taxon_name:N', sort='-x'),
 alt.Color('taxon_name', title='Taxon name')
).properties(width=500, height=500)

alt.Chart(...)

### Pie chart of abundances

In [288]:
(clean_kraken(kraken_megahit_raw)
 .loc[lambda x: ~x['species'].isna()]
 .loc[lambda x: x['count'] > 100]
 .sort_values('count', ascending=False)
 .assign(percent=lambda x: x['count'] / x['count'].sum())
)

kraken_cleaned = (clean_kraken(kraken_megahit_raw)
                     .loc[lambda x: x['taxonomy'].str.contains('d__')])

pie = alt.Chart(kraken_cleaned).mark_arc(innerRadius=50).encode(
 theta=alt.Theta(field="count", type="quantitative"),
 color=alt.Color(field="taxonomy", type="nominal"),
).properties(width=500)

bar = alt.Chart(kraken_cleaned).mark_bar().encode(
 alt.X('taxonomy:N', sort='-y'),
 alt.Y('count:Q'),
 alt.Color('taxonomy:N')
).properties(width=500)

alt.vconcat(pie, bar)

alt.VConcatChart(...)

## Reading in results from CAT from nf-core
### quite some cleaning up 

In [276]:
cat = pd.read_csv('MEGAHIT-MaxBin2-CAPES_S11.bin2classification.names.txt',
                  sep='\t', 
                  skiprows=2,
                  header=None,
                  usecols=[0, 1, 2, 3, 8, 9, 10, 11, 12, 13, 14, 15],
                  names=['bin', 'classification', 'num_ORF',
                         'num_ORF_classification_based', 'kingdom', 
                         'clade', 'clade_phylum', 'phylum_class', 'class_order', 
                         'order_family', 'family_genus', 'genus'])
                 
   

In [289]:
cat.head(5)

,bin,classification,num_ORF,num_ORF_classification_based,kingdom,clade,clade_phylum,phylum_class,class_order,order_family,family_genus,genus
0,MEGAHIT-MaxBin2-CAPES_S11.002.fa,classified,1455,1442,Bacteria (superkingdom): 0.97,FCB group (clade): 0.74,Bacteroidetes/Chlorobi group (clade): 0.74,Bacteroidetes (phylum): 0.74,Bacteroidia (class): 0.74,Bacteroidales (order): 0.74,Bacteroidaceae (family): 0.65,Bacteroides (genus): 0.65
1,MEGAHIT-MaxBin2-CAPES_S11.003.fa,classified,1790,1782,Bacteria (superkingdom): 0.97,Terrabacteria group (clade): 0.82,Actinobacteria (phylum): 0.71,Actinobacteria (class): 0.71,Bifidobacteriales (order): 0.71,Bifidobacteriaceae (family): 0.71,Bifidobacterium (genus): 0.71,NaN
2,MEGAHIT-MaxBin2-CAPES_S11.004.fa,classified,2022,2021,Bacteria (superkingdom): 0.72,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MEGAHIT-MaxBin2-CAPES_S11.005.fa,classified,6074,6024,Bacteria (superkingdom): 0.98,FCB group (clade): 0.46,Bacteroidetes/Chlorobi group (clade): 0.46,Bacteroidetes (phylum): 0.46,Bacteroidia (class): 0.46,Bacteroidales (order): 0.46,Bacteroidaceae (family): 0.40,Bacteroides (genus): 0.40
4,MEGAHIT-MaxBin2-CAPES_S11.006.fa,classified,2721,2717,Bacteria (superkingdom): 0.83,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [385]:
(cat
 .melt(id_vars=['bin', 'classification', 'num_ORF', 
                'num_ORF_classification_based'])
 .loc[lambda x: ~x['value'].isna()]
 .drop(columns='variable')
 #.value_counts('bin')
 #.loc[lambda x: x['bin'] == 'MEGAHIT-MaxBin2-CAPES_S11.002.fa']
 .assign(certainty=lambda x: x['value'].str.extract(r'(\d+.*)').astype(float),
         name=lambda x: x['value'].str.extract(r'(.*?)\('),
         order=lambda x: x['value'].str.extract(r'.*\((.*?)\)'))
 .loc[lambda x: x['order'] != 'clade']
 
)

,bin,classification,num_ORF,num_ORF_classification_based,value,certainty,name,order
0,MEGAHIT-MaxBin2-CAPES_S11.002.fa,classified,1455,1442,Bacteria (superkingdom): 0.97,0.97,Bacteria,superkingdom
1,MEGAHIT-MaxBin2-CAPES_S11.003.fa,classified,1790,1782,Bacteria (superkingdom): 0.97,0.97,Bacteria,superkingdom
2,MEGAHIT-MaxBin2-CAPES_S11.004.fa,classified,2022,2021,Bacteria (superkingdom): 0.72,0.72,Bacteria,superkingdom
3,MEGAHIT-MaxBin2-CAPES_S11.005.fa,classified,6074,6024,Bacteria (superkingdom): 0.98,0.98,Bacteria,superkingdom
4,MEGAHIT-MaxBin2-CAPES_S11.006.fa,classified,2721,2717,Bacteria (superkingdom): 0.83,0.83,Bacteria,superkingdom
...,...,...,...,...,...,...,...,...
231,MEGAHIT-MaxBin2-CAPES_S11.009.fa,classified,4822,4781,Bacteroides (genus): 0.69,0.69,Bacteroides,genus
238,MEGAHIT-MaxBin2-CAPES_S11.016.fa,classified,2318,2280,Acidaminococcus massiliensis (species): 0.49,0.49,Acidaminococcus massiliensis,species
244,MEGAHIT-MaxBin2-CAPES_S11.022.fa,classified,2097,1994,unclassified Olsenella (no rank): 0.82,0.82,unclassified Olsenella,no rank
250,MEGAHIT-MaxBin2-CAPES_S11.028.fa,classified,6336,6098,Bacteroides (genus): 0.59,0.59,Bacteroides,genus


### CAT LCA file

### maybe the lca file is nothing to have? seems to be redundant if you have the bin2classificaiotn file. 

## Reading in results from gtdb-tk from nf-core (the summary tsv)

In [492]:
gtdb_raw = pd.read_csv('gtdbtk_summary.tsv',
                  sep='\t')

def massage_gtdb(gtwb_raw: pd.DataFrame, 
                 order: str, 
                 sample_name: str, 
                 assembly_type: str) -> pd.DataFrame:
    
    rank = {'order': 'o__', 'family': 'f__'}
    
    return (gtdb_raw
     .loc[lambda x: x['user_genome'].str.contains(f'(?=.*{sample_name})(?=.*{assembly_type})')]
     .loc[lambda x: ~x['classification'].isna()]
     .assign(classification=lambda x: x['classification'].str.split(';'))
     .explode(column='classification')
     .loc[lambda x: x['classification'].str.contains(rank[order])]
     .assign(classification=lambda x: x['classification'].str.replace(rank[order], ''))
    )
    
gtdb = massage_gtdb(gtdb_raw, 'order', '_S11', 'MEGA')


In [489]:
alt.Chart(gtdb).mark_bar().encode(
 alt.X('count(classification):Q'),
 alt.Y('classification:N', sort='-x')
)

alt.Chart(...)

### Centrifuge classificaiton (right after preprocessing)

### needs a way to identify which one is virus. Maybe this is sorted out if one uses another db to classify?

In [185]:
centrifuge = pd.read_csv('/Users/williamrosenbaum/Downloads/report.txt',
                         sep='\t')

In [406]:
(centrifuge
 .loc[lambda x: x['abundance'] > 0]
 .sort_values(by='abundance', ascending=False)
# .loc[lambda x: x['name'].str.contains(r'virus|phage')] #try to filter out virus 
)

,name,taxID,taxRank,genomeSize,numReads,numUniqueReads,abundance
186,Klebsiella pneumoniae,573,species,15713353,4403776,3921236,2.082890e-01
179,Escherichia coli,562,species,12319210,2998448,1302439,2.055870e-01
1913,Bifidobacterium longum,216816,species,3655960,735601,695952,1.486330e-01
1513,Streptococcus infantarius,102684,species,1913271,309029,264686,7.275100e-02
269,Bacteroides thetaiotaomicron,818,species,6390542,977349,942527,7.028640e-02
...,...,...,...,...,...,...,...
2420,Dehalogenimonas lykanthroporepellens,552810,species,1686510,77,49,1.403740e-177
1983,Fictibacillus arsenicus,255247,species,4055461,49,39,1.128970e-188
2824,Fictibacillus phosphorivorans,1221500,species,4230665,95,73,1.057440e-189
1451,Bacillus weihenstephanensis,86662,species,5740546,135,42,1.123280e-204


## Genome binning summary (/results/binning/summary)

In [407]:
binning = pd.read_csv('bin_summary.tsv',
                      sep='\t')

In [217]:
(binning
 .iloc[:, 1:]
)

,bin,Depth CAPES_S11,Depth CAPES_S21,Depth CAPES_S7,GenomeBin,Domain,%Complete (domain),%Complete and single-copy (domain),%Complete and duplicated (domain),%Fragmented (domain),...,closest_placement_ani,closest_placement_af,pplacer_taxonomy,classification_method,note,"other_related_references(genome_id,species_name,radius,ANI,AF)",msa_percent,translation_table,red_value,warnings
0,MEGAHIT-MaxBin2-CAPES_S7.002.fa,12.13450,165.35450,65.374600,MEGAHIT-MaxBin2-CAPES_S7.002.fa,bacteria_odb10,92.7,91.1,1.6,0.8,...,98.73,0.89,d__Bacteria;p__Actinobacteriota;c__Actinomycet...,taxonomic classification defined by topology a...,topological placement and ANI have congruent s...,"GCF_000269965.1, s__Bifidobacterium infantis, ...",90.27,11.0,NaN,NaN
1,MEGAHIT-MaxBin2-CAPES_S7.003.fa,23.78450,77.88510,77.397200,MEGAHIT-MaxBin2-CAPES_S7.003.fa,bacteria_odb10,97.6,83.1,14.5,1.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MEGAHIT-MaxBin2-CAPES_S7.005.fa,1.24615,3.89888,123.401000,MEGAHIT-MaxBin2-CAPES_S7.005.fa,bacteria_odb10,58.8,54.8,4.0,2.4,...,99.03,0.88,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,taxonomic classification defined by topology a...,topological placement and ANI have congruent s...,"GCF_000828055.2, s__Klebsiella variicola, 95.0...",62.16,11.0,NaN,NaN
3,MEGAHIT-MaxBin2-CAPES_S7.001.fa,4.20057,113.81500,122.391000,MEGAHIT-MaxBin2-CAPES_S7.001.fa,bacteria_odb10,90.3,77.4,12.9,2.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MEGAHIT-MaxBin2-CAPES_S7.004.fa,48.09430,4.50323,115.534000,MEGAHIT-MaxBin2-CAPES_S7.004.fa,bacteria_odb10,63.7,52.4,11.3,10.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
578,SPAdes-MaxBin2-CAPES_S7.008.fa,1.16163,0.00000,29.996900,SPAdes-MaxBin2-CAPES_S7.008.fa,bacteria_odb10,95.1,91.1,4.0,1.6,...,NaN,NaN,d__Bacteria;p__Firmicutes_C;c__Negativicutes;o...,taxonomic classification defined by topology a...,NaN,"GCF_902810435.1, s__Veillonella parvula_A, 95....",88.90,11.0,0.996764,Genome not assigned to closest species as it f...
579,SPAdes-MaxBin2-CAPES_S7.010.fa,0.00000,0.00000,13.498000,SPAdes-MaxBin2-CAPES_S7.010.fa,bacteria_odb10,42.7,41.9,0.8,8.9,...,95.44,0.59,d__Bacteria;p__Firmicutes;c__Bacilli;o__Lactob...,ANI,topological placement and ANI have incongruent...,"GCF_900478025.1, s__Streptococcus pasteurianus...",50.94,11.0,NaN,NaN
580,SPAdes-MaxBin2-CAPES_S7.011.fa,0.00000,0.00000,10.866700,SPAdes-MaxBin2-CAPES_S7.011.fa,bacteria_odb10,26.6,24.2,2.4,8.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
581,SPAdes-MaxBin2-CAPES_S7.012.fa,0.00000,0.00000,8.468365,SPAdes-MaxBin2-CAPES_S7.012.fa,bacteria_odb10,76.6,75.0,1.6,11.3,...,98.64,0.71,d__Bacteria;p__Firmicutes_A;c__Clostridia;o__L...,taxonomic classification defined by topology a...,topological placement and ANI have congruent s...,"GCA_900546325.1, s__Faecalimonas sp900546325, ...",71.71,11.0,NaN,NaN


## Making histogram of the contig lengths

### megahit and facet by category (i.e. long, short... )

In [688]:
import pandas as pd
import altair as alt
import pathlib
import numpy as np
alt.data_transformers.disable_max_rows()


contig_file = pathlib.Path('../results/megahit/Bat-Guano-15_S6_L001_R.csv')
contigs = pd.read_csv(contig_file)




### If the contigs spans a broad range, do the graph and manipulation below:

In [689]:
(contigs
 .assign(category=pd.cut(contigs.length, bins=[1, 1000, 50000, np.inf], labels=['short', 'medium', 'long']))
)


number_contigs = contigs.name.nunique()

charts = []
for cat in contigs.category.unique():
    chart = alt.Chart(contigs.loc[lambda x: x['category'] == cat], 
                      title=f'MEGAHIT. Contigs of {contig_file.name}. {cat.title()}').mark_bar().encode(
         alt.X('length', bin=True, title='Length (bp)'),
         alt.Y('count()', title='# of contigs'))
    charts.append(chart)
    
    
plot = alt.hconcat(*charts)
plot

AttributeError: 'DataFrame' object has no attribute 'category'

### Otherwise, do

In [690]:
alt.Chart(contigs).mark_bar().encode(
    alt.X('length:Q', bin=alt.Bin(step=1000)),
    alt.Y('count():Q')
)

alt.Chart(...)

### metaspades

In [144]:
metaspades = pd.read_csv('../results/metaspades/APX_contigs.csv')

In [146]:
metaspades.sort_values('length', ascending=False)

,length,name
0,502124,1
1,353595,2
2,305353,3
3,250474,4
4,247421,5
...,...,...
673233,56,673234
673234,56,673235
673235,56,673236
673236,56,673237


### Looking at my BAT file

In [691]:
bat = pd.read_csv('../results/cat/CAT_APX_names.txt',
                  sep='\t')
                  #skiprows=1)
                  #header=None)
                  #usecols=[0, 1, 2, 3, 8, 9, 10, 11, 12, 13, 14, 15],
                  #names=['bin', 'classification', 'num_ORF',
                      #   'num_ORF_classification_based', 'kingdom', 
                      #   'clade', 'clade_phylum', 'phylum_class', 'class_order', 
                      #   'order_family', 'family_genus', 'genus'])

(bat
# .melt(id_vars=['bin', 'classification'])
 #.loc[lambda x: ~x['value'].isna()]
 #.drop(columns='variable')
 #.value_counts('bin')
 #.loc[lambda x: x['bin'] == 'MEGAHIT-MaxBin2-CAPES_S11.002.fa']
 #.assign(certainty=lambda x: x['value'].str.extract(r'(\d+.*)').astype(float),
 #        name=lambda x: x['value'].str.extract(r'(.*?)\('),
 #        order=lambda x: x['value'].str.extract(r'.*\((.*?)\)'))
 #.loc[lambda x: x['order'] != 'clade']
 
)

FileNotFoundError: [Errno 2] No such file or directory: '../results/cat/CAT_APX_names.txt'

### sorting fastq files with pyfastx

In [20]:
import pyfastx

fastq1 = pyfastx.Fastq('../testdata_dolphin/Dol1_S19_L001_R1_001.fastq.gz')
fastq2 = pyfastx.Fastq('../testdata_dolphin/Dol1_S19_L001_R2_001.fastq.gz')

### Bracken report

In [695]:
bracken_raw = pd.read_csv('../results/kraken/Bat-Guano-15_S6_L001_R.kraken2.report_bracken_species.tsv', 
                      sep='\t', 
                      header=None, 
                      names=['percent', 'new_est_reads', 'unknown', 
                             'level', 'tax_id', 'name'])


bracken = (bracken_raw
 .drop(columns=['unknown'])
 .assign(percent=lambda x: x['percent'] / 100, 
         domain=lambda x: np.select([x['name'].str.contains('Viruses'),
                                     x['name'].str.contains('Bacteria'), 
                                     x['name'].str.contains('Eukaryota')],
                                                ['Virus', 'Bacteria', 'Eukaryota'], default=pd.NA))
 .assign(domain=lambda x: x['domain'].fillna(method='ffill'))
 .dropna()
 .loc[lambda x: x['level'] == 'S']
)

bracken

,percent,new_est_reads,level,tax_id,name,domain
10,0.6828,52138,S,1933187,Rift Valley fever phlebovirus,Virus
19,0.0074,568,S,28295,Porcine epidemic diarr...,Virus
20,0.0058,446,S,693999,Scotophilus bat corona...,Virus
22,0.0015,111,S,2501928,Nyctalus velutinus alp...,Virus
24,0.0012,95,S,2501929,NL63-related bat coron...,Virus
26,0.0012,93,S,1508224,Lucheng Rn rat coronav...,Virus
28,0.0008,62,S,1913642,Mink coronavirus 1,Virus
30,0.0006,44,S,694000,Miniopterus bat corona...,Virus
32,0.0002,18,S,2501927,Myotis ricketti alphac...,Virus
34,0.0001,11,S,1913643,Bat coronavirus CDPHE15,Virus


# KAIJU2NAMES!


In [741]:

kaiju2names = pd.read_csv('../results/kaiju/megahit/Bat-Guano-15_S6_L001_R_names_megahit.out', 
                          sep='\t', 
                          header=None,
                          index_col=False,
                          names=['classfied', 'name', 
                                 'NCBI', 'match_length', 
                                 'all_matches', 'aa_matches', 
                                 'aa_sequence', 'species']).loc[:, ['name', 'match_length', 'aa_sequence', 'species', 'NCBI']]


megahit_contigs = pd.read_csv('~/virusclass/results/megahit/Bat-Guano-15_S6_L001_R.csv')
    


merged_raw = kaiju2names.merge(megahit_contigs, on='name').sort_values('length', ascending=False)




merged = (merged_raw
    .dropna()
    .assign(summed=lambda x: x.groupby('species')['length'].transform(sum) / x['length'].sum())
    .assign(species=lambda x: x['species'].str.split(';').str[:-1])
    .assign(last_level_kaiju=lambda x: x['species'].str[-1])
    .assign(second_level_kaiju=lambda x: x['species'].str[-2])
    .assign(third_level_kaiju=lambda x: x['species'].str[-3])

    .assign(kingdom_kaiju=lambda x: np.select([x['species'].str[0] != 'cellular organisms'],
                                        [x['species'].str[0]],
                                        default=x['species'].str[1]))
)

taxonomy_list = ['kingdom', 'phylum', 'class', 'order', 'family', 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

(merged_raw
 .assign(species=lambda x: x['species'].str.split(';').str[:-1])
 
 .assign(species=lambda x: np.select([x['species'].str[0] != 'cellular organisms'],
                                     [x['species']],
                                     default=x['species'].str[1:]))
 .explode('species')
 .rename(columns={'species': 'rank'})
 .dropna()
 .assign(taxonomy=lambda x: x.groupby('name')['name'].transform(lambda x: taxonomy_list[:x.count()]))
)

merged

,name,match_length,aa_sequence,species,NCBI,length,sequence,summed,last_level_kaiju,second_level_kaiju,third_level_kaiju,kingdom_kaiju
18,k79_50,751,YRYNRPTVLDICQARVVYQIVQRYFDIYEGGCITAKEVVVTNLNKS...,"[Viruses, Riboviria, Orthornavirae, Pisuvir...",28295,13264,TGCGTACATTATAAACATCTTTATCACAAAACCACGGATCAGTGAT...,0.414768,Porcine epidemic diarrhea virus,Pedacovirus,Alphacoronavirus,Viruses
19,k79_49,238,"QGTSKSFYVHANGGSKFCKKHNFFCLNCDSYGPGCTFINDVIA,","[Viruses, Riboviria, Orthornavirae, Pisuvir...",28295,7323,CTAAAATCAACATTTAATGATGCAAGCAATGTTGAATCAACTAATT...,0.414768,Porcine epidemic diarrhea virus,Pedacovirus,Alphacoronavirus,Viruses
14,k79_44,263,ILTILLVVLQYGHYKYSRVLYGLKMAILWLLWPLVLALSIFDAWAS...,"[Viruses, Riboviria, Orthornavirae, Pisuvir...",693999,6772,TCCATATCAACACCGCATAGTCTTCAGTTACTAAGCAGTTCAAGAC...,0.136436,Scotophilus bat coronavirus 512,Pedacovirus,Alphacoronavirus,Viruses
15,k79_47,7556,YMGKLSLLTLLEDKAATEELQTIARYIIMEGFVSPPEIPKPHKMTS...,"[Viruses, Riboviria, Orthornavirae, Negarna...",11588,6534,TATAAACCATCTCCTCTACTTCAGAGTCCTCTAGCTCCCGATTAGA...,0.245694,Rift Valley fever virus,Rift Valley fever phlebovirus,Phlebovirus,Viruses
13,k79_45,2332,EDPHLRNRPGKGHNYIDGMTQEDATCKPVTYAGACSSFDVLLEKGK...,"[Viruses, Riboviria, Orthornavirae, Negarna...",11588,3871,GGTGCATTAAATGTATGTTTTATTAACAATTCTAATCTCGGTTCTG...,0.245694,Rift Valley fever virus,Rift Valley fever phlebovirus,Phlebovirus,Viruses
12,k79_39,1378,MIEQDGLHAGSPAAWVERLFGYDWAQQTIGCSDAAVFRLSAQGRPV...,"[cellular organisms, Bacteria]",2,3749,GCCACCACGCCCAGCTAATTTTTTGTATTTTTAGTAGAGATGGGGT...,0.075531,Bacteria,cellular organisms,NaN,Bacteria
11,k79_40,1256,MDNYQELAIQFAAQAVDRNEIEQWVREFAYQGFDARRVIELLKQYG...,"[Viruses, Riboviria, Orthornavirae, Negarna...",11588,1790,CATAGAATAAGGTATCCTGGGAGGACCATCTCCTCTAAAGTACTCC...,0.245694,Rift Valley fever virus,Rift Valley fever phlebovirus,Phlebovirus,Viruses
17,k79_42,1145,VTAYEQTVNQEINFQKSFNNHVRTLVAREEVAIGLMHLWDFVSALE...,"[Viruses, Varidnaviria, Bamfordvirae, Prepl...",129951,1332,GGTCCGGCCCCAGCTGCCTCCAGGGCGCGTCGGCTTGGGGCCCAGC...,0.102246,Human mastadenovirus C,Mastadenovirus,Adenoviridae,Viruses
3,k79_15,1068,IAPFTDSGSVSRDTYLGHLLTLYREAIGQAHVDEHTFQEITSVSRA...,"[Viruses, Varidnaviria, Bamfordvirae, Prepl...",129951,800,CTTTCCTTTCGCAGCGCCGCCTCTGCCTGCTCGCGCTGTTGCAACT...,0.102246,Human mastadenovirus C,Mastadenovirus,Adenoviridae,Viruses
4,k79_26,261,MHEPPVQPDRCALSGNYRLESNPVRHDLSPLAAATGNRISRARYVG...,"[cellular organisms, Bacteria, Proteobacteri...",553,737,CGCTGAGATAGGTGCCTCACTGATTAAGCATTGGTAACTGTCAGAC...,0.014848,Pantoea ananatis,Pantoea,Erwiniaceae,Bacteria


## Cleaning the CAT on contigs file

In [750]:
cat_raw = pd.read_csv('../results/cat/CAT_Bat-Guano-15_S6_L001_R_contigs_names.txt',
                  sep='\t')
cat = (cat_raw
    .loc[lambda x: x['classification'] != 'no taxid assigned']
    .loc[lambda x: x['superkingdom'] != 'no support']
    .loc[lambda x: x['phylum'] != 'no support']
    .drop(columns=['classification', 'lineage', 'lineage scores'])
       
    .rename(columns={'# contig': 'name',
                     'species': 'last_level',
                     'genus': 'second_level', 
                     'family': 'third_level'})
       
    .assign(kingdom_cat=lambda x: x['superkingdom'].str[:-6])
)

supermerged = merged.merge(cat, on='name', how='left')
supermerged[['name', 'length', 'last_level_kaiju', 
             'second_level_kaiju', 'kingdom_kaiju', 'phylum', 
             'class', 'order', 'last_lev', 'kingdom_cat']]

,name,length,last_level_kaiju,second_level_kaiju,kingdom_kaiju,phylum,class,order,species,kingdom_cat
0,k79_50,13264,Porcine epidemic diarrhea virus,Pedacovirus,Viruses,Pisuviricota: 1.00,Pisoniviricetes: 1.00,Nidovirales: 1.00,"[Viruses, Riboviria, Orthornavirae, Pisuvir...",Viruses
1,k79_49,7323,Porcine epidemic diarrhea virus,Pedacovirus,Viruses,Pisuviricota: 1.00,Pisoniviricetes: 1.00,Nidovirales: 1.00,"[Viruses, Riboviria, Orthornavirae, Pisuvir...",Viruses
2,k79_44,6772,Scotophilus bat coronavirus 512,Pedacovirus,Viruses,Pisuviricota: 1.00,Pisoniviricetes: 1.00,Nidovirales: 1.00,"[Viruses, Riboviria, Orthornavirae, Pisuvir...",Viruses
3,k79_47,6534,Rift Valley fever virus,Rift Valley fever phlebovirus,Viruses,Negarnaviricota: 1.00,Ellioviricetes: 1.00,Bunyavirales: 1.00,"[Viruses, Riboviria, Orthornavirae, Negarna...",Viruses
4,k79_45,3871,Rift Valley fever virus,Rift Valley fever phlebovirus,Viruses,NaN,NaN,NaN,"[Viruses, Riboviria, Orthornavirae, Negarna...",NaN
5,k79_39,3749,Bacteria,cellular organisms,Bacteria,NaN,NaN,NaN,"[cellular organisms, Bacteria]",NaN
6,k79_40,1790,Rift Valley fever virus,Rift Valley fever phlebovirus,Viruses,NaN,NaN,NaN,"[Viruses, Riboviria, Orthornavirae, Negarna...",NaN
7,k79_42,1332,Human mastadenovirus C,Mastadenovirus,Viruses,NaN,NaN,NaN,"[Viruses, Varidnaviria, Bamfordvirae, Prepl...",NaN
8,k79_15,800,Human mastadenovirus C,Mastadenovirus,Viruses,NaN,NaN,NaN,"[Viruses, Varidnaviria, Bamfordvirae, Prepl...",NaN
9,k79_26,737,Pantoea ananatis,Pantoea,Bacteria,NaN,NaN,NaN,"[cellular organisms, Bacteria, Proteobacteri...",NaN


# Samtools mpileup coverage

In [831]:
coverage = pd.read_csv('../results/samtools/coverage_Bat-Guano-15_S6_L001_R.tsv',
                       sep='\t', 
                       header=None, 
                       usecols=[0,1,3],
                       names=['contig', 'position', 'coverage'])

#(coverage
# .assign(median_coverage = lambda x: x.groupby('contig', as_index=False)['coverage'].transform(lambda x: x.median()))
# .assign(mean_coverage = lambda x: x.groupby('contig', as_index=False)['coverage'].transform(lambda x: x.mean()))
#
#)

In [856]:
plots = []

for contig in coverage.contig.unique():
    
    base = alt.Chart(
    coverage
     .loc[lambda x: x.contig == contig],
     title=f'Coverage of {contig}. Red line shows mean.'
    ).mark_line().encode(
     alt.X('position:N', axis=alt.Axis(values=np.arange(0, 50000, 100))),
     alt.Y('coverage:Q')
    ).properties(width=1000)
    
    rule = alt.Chart(
    coverage
     .loc[lambda x: x.contig == contig],
     title=f'Coverage of {contig}'
    ).mark_rule(color='red').encode(
     alt.Y('mean(coverage):Q')
    ).properties(width=1000)
    
    plot = (base + rule).encode(alt.Y(axis=alt.Axis(title='Reads coverage')))
    plots.append(plot)

In [861]:
plots[15]

alt.LayerChart(...)

# WWW BLAST with biopython

In [538]:
# to slow...

# Scripts

In [505]:
%%writefile bin/config_length_megahit.py
#!/usr/bin/env python

import pandas as pd
import fire

def extract_length(file: str, name: str) -> None:
    '''
    Saves a .csv file with the length and name of the contigs from the megahit. 
    
    file: The fasta file of contigs from megahit
    name: The name of the .csv file (preferable the sample_id)
    '''
    
    with open(file, 'r') as fasta:
        content = fasta.readlines()
    
                                  
    (pd.DataFrame()
          .assign(info=[x.strip() for x in content if x.startswith('>')],
                  name=lambda x: x['info'].str.extract(r'>(.*?) flag'),
                  length=lambda x: x['info'].str.extract(r'len=(\d*)').astype(int),
                  sequence=[x.strip() for x in content if not x.startswith('>')])
          .drop(columns=['info'])
          .sort_values('length', ascending=False)
          .to_csv(f'{name}.csv', index=False)
    )
    

if __name__ == '__main__':
    fire.Fire(extract_length)


Overwriting bin/config_length_megahit.py


In [418]:
%%writefile bin/config_length_metaspades.py
#!/usr/bin/env python

import pandas as pd
import fire

def extract_length(file: str, name: str) -> None:
    '''
    Saves a .csv file with the length and name of the contigs from the metaspades assembly. 
    
    file: The fasta file of contigs from megahit
    name: The name of the .csv file (preferable the sample_id)
    '''
    
    with open(file, 'r') as fasta:
        df = pd.DataFrame([line.strip() for line in fasta.readlines() if line.startswith('>')],
                          columns=['line'])
        
    (df
     .assign(length=lambda x: x['line'].str.extract(r'length_(\d*)').astype(int),
             name=lambda x: x['line'].str.extract(r'NODE_(\d*)'))
     .loc[:, ['length', 'name']]
     .assign(name=lambda x: x['name'].str.strip())
     .sort_values('length', ascending=False)
     .to_csv(f'{name}.csv', index=False)
    )
    

if __name__ == '__main__':
    fire.Fire(extract_length)


Overwriting bin/config_length_metaspades.py


In [419]:
%%writefile bin/rename_contigs.py
#!/usr/bin/env python

import fire

def rename_contigs(file: str) -> None:
    '''
    Rename the files of contigs from megahit, to look like: >contig_1, >contig_2 etc. 
    
    file: The fasta file of contigs from megahit
    '''
    
    with open(file, 'r') as fasta:
        content = [line.strip() for line in fasta.readlines()]
    
    #write new file
    num = 1
    with open(file, 'w') as new_fasta:
        for line in content:
            if line.startswith('>'):
                line = f'>contig_{num}'
                num += 1
            print(line, file=new_fasta)


if __name__ == '__main__':
    fire.Fire(rename_contigs)

Overwriting bin/rename_contigs.py
